In [3]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Activation, LSTM, Dropout
from keras.metrics import categorical_accuracy
import numpy as np

home_dir = './'

x_train = np.load(home_dir + 'data/mbe_3sec_random_default.npy')
y_train = np.load(home_dir + 'data/y.npy')

x_test = np.load(home_dir + 'data/mbe_3sec_random_default_test.npy')
y_test = np.load(home_dir + 'data/y_test.npy')

x_unknown = np.load(home_dir + 'data/x_unknown.npy')


In [16]:

model = Sequential()

model.add(LSTM(256, input_shape = (x_train.shape[1:]), return_sequences = False))
model.add(Dropout(0.6))
model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])


Train on 10176 samples, validate on 1131 samples
Epoch 1/100


  128/10176 [..............................] - ETA: 2:37 - loss: 0.4317 - categorical_accuracy: 0.8672

  256/10176 [..............................] - ETA: 2:07 - loss: 0.7193 - categorical_accuracy: 0.7930

  384/10176 [>.............................] - ETA: 1:56 - loss: 0.7316 - categorical_accuracy: 0.7734

  512/10176 [>.............................] - ETA: 1:50 - loss: 0.8213 - categorical_accuracy: 0.7480

...

  768/10176 [=>............................] - ETA: 1:43 - loss: 0.7935 - categorical_accuracy: 0.7565

  896/10176 [=>............................] - ETA: 1:40 - loss: 0.8268 - categorical_accuracy: 0.7455

 1024/10176 [==>...........................] - ETA: 1:38 - loss: 0.8753 - categorical_accuracy: 0.7334

 1152/10176 [==>...........................] - ETA: 1:36 - loss: 0.8975 - categorical_accuracy: 0.7240

KeyboardInterrupt: 

In [18]:

model.fit(x_train, y_train, batch_size=128, validation_split=0.1, epochs = 100, callbacks=[])


Train on 10176 samples, validate on 1131 samples
Epoch 1/100


  128/10176 [..............................] - ETA: 2:37 - loss: 0.4317 - categorical_accuracy: 0.8672

  256/10176 [..............................] - ETA: 2:07 - loss: 0.7193 - categorical_accuracy: 0.7930

  384/10176 [>.............................] - ETA: 1:56 - loss: 0.7316 - categorical_accuracy: 0.7734

  512/10176 [>.............................] - ETA: 1:50 - loss: 0.8213 - categorical_accuracy: 0.7480

...

  768/10176 [=>............................] - ETA: 1:43 - loss: 0.7935 - categorical_accuracy: 0.7565

  896/10176 [=>............................] - ETA: 1:40 - loss: 0.8268 - categorical_accuracy: 0.7455

 1024/10176 [==>...........................] - ETA: 1:38 - loss: 0.8753 - categorical_accuracy: 0.7334

 1152/10176 [==>...........................] - ETA: 1:36 - loss: 0.8975 - categorical_accuracy: 0.7240

KeyboardInterrupt: 

In [17]:

model.load_weights('models/LSTM_1')

Train on 10176 samples, validate on 1131 samples
Epoch 1/100


  128/10176 [..............................] - ETA: 2:37 - loss: 0.4317 - categorical_accuracy: 0.8672

  256/10176 [..............................] - ETA: 2:07 - loss: 0.7193 - categorical_accuracy: 0.7930

  384/10176 [>.............................] - ETA: 1:56 - loss: 0.7316 - categorical_accuracy: 0.7734

  512/10176 [>.............................] - ETA: 1:50 - loss: 0.8213 - categorical_accuracy: 0.7480

...

  768/10176 [=>............................] - ETA: 1:43 - loss: 0.7935 - categorical_accuracy: 0.7565

  896/10176 [=>............................] - ETA: 1:40 - loss: 0.8268 - categorical_accuracy: 0.7455

 1024/10176 [==>...........................] - ETA: 1:38 - loss: 0.8753 - categorical_accuracy: 0.7334

 1152/10176 [==>...........................] - ETA: 1:36 - loss: 0.8975 - categorical_accuracy: 0.7240

KeyboardInterrupt: 

In [15]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(model.predict(x_test), axis=1)))


             precision    recall  f1-score   support

          0       0.61      0.95      0.74        41
          1       0.71      0.74      0.73        50
          2       0.24      0.92      0.37        49
          3       0.73      0.64      0.68        47
          4       0.85      0.76      0.80        59
          5       0.62      0.14      0.23        72
          6       1.00      0.07      0.13       141
          7       0.22      0.71      0.33        14

avg / total       0.73      0.48      0.43       473



In [13]:
names = np.loadtxt(home_dir + 'meta/filenames_test.txt', dtype='str')

text_labels = {'0': 'background', '1': 'bags', '2': 'door', '3': 'keyboard', '4': 'knocking', '5': 'ring',
               '6': 'speech', '7': 'tool'}
result_predict = model.predict(x_test)
result = np.vstack((names, np.around(np.amax(result_predict, axis=1), decimals=3),
                    [text_labels[str(np.argmax(res))] for res in result_predict])).T

np.savetxt(home_dir + 'result.txt', result, fmt='%s')


(473,)


In [20]:
names = np.loadtxt(home_dir + 'meta/filenames_unknown.txt', dtype='str')

result_predict = model.predict(x_unknown)
result = np.vstack((names, np.around(np.amax(result_predict, axis=1), decimals=3),
                    [text_labels[str(np.argmax(res))] for res in result_predict])).T

np.savetxt(home_dir + 'result-unknown.txt', result, fmt='%s')


In [0]:
model.save(home_dir + 'LSTM_2')